In [1]:
import datetime
from netCDF4 import Dataset,num2date
import numpy as np
import pandas as pd
import glob

In [ ]:
path = 'temperature data/tmax*'
file_path_tmax = []
for file in glob.iglob(path, recursive = True):
    file_path_tmax.append(file)
file_path_tmax.sort()
file_path_tmax

In [ ]:
path = 'temperature data/tmin*'
file_path_tmin = []
for file in glob.iglob(path, recursive = True):
    file_path_tmin.append(file)
file_path_tmin.sort()
file_path_tmin

In [ ]:
#dataset = Dataset("temperature data/tmax.2001.nc")

def data_Preprocessing(dataset):

    nctime = dataset.variables['time'][:] # get values
    t_unit = dataset.variables['time'].units # get unit  "days since 1900-01-01T00:00:00Z"
    tvalue = num2date(nctime,units = t_unit)
    str_time = [i.strftime("%Y-%m-%d") for i in tvalue] # to display dates as string

    longitude_values = np.repeat(list(dataset['lon']), len(list(dataset['lat']))*len(list(dataset['time'])))
    latitude_values = list(dataset['lat'])*len(list(dataset['lon']))*len(list(dataset['time']))
    time = list(str_time)*len(list(dataset['lon']))*len(list(dataset['lat']))
    temperature_values = np.array(list(dataset['tmax'])).flatten()

    ds = pd.DataFrame({"lon": longitude_values, "lat": latitude_values, "time": time, "tmax": temperature_values})
    ds['time'] = pd.to_datetime(ds['time'])
    ds['month'] = ds['time'].dt.month
    ds['tmax'] = ds['tmax'].mask(ds['tmax'] == ds['tmax'].iloc[0], np.nan)
    
    ds1 = ds.loc[ds['month'] == 1]
    ds2 = ds.loc[ds['month'] == 2]
    ds3 = ds.loc[ds['month'] == 3]
    ds4 = ds.loc[ds['month'] == 4]
    ds5 = ds.loc[ds['month'] == 5]
    
    return ds1, ds2, ds3, ds4, ds5

# calculate temperature

In [ ]:
def get_tmax(dataset):
    df_selected = dataset[((dataset['lon'] <= 265.38175) & 
                           (dataset['lon'] >= 255.94595) &
                           (dataset['lat'] >= 36.99900) &
                           (dataset['lat'] <= 41.00132)) |   
                          #Kansas Area
                          
                          ((dataset['lon'] <= 262.71654) & 
                           (dataset['lon'] >= 256.98720) &
                           (dataset['lat'] <= 36.99900) &
                           (dataset['lat'] >= 33.89811)) |
                          #Oklahoma Area
                          
                          ((dataset['lon'] <= 255.94281) & 
                           (dataset['lon'] >= 247.94240) &
                           (dataset['lat'] <= 48.99742) &
                           (dataset['lat'] >= 44.99552)) |
                          #Montana Area
                          
                          ((dataset['lon'] >= 255.94018) & 
                           (dataset['lon'] <= 257.93342) &
                           (dataset['lat'] <= 43.00957) &
                           (dataset['lat'] >= 41.00132))
                          #Nebraska Area
                          
                         ].reset_index(drop = True)
    
    df_selected['tmax'].fillna(df_selected['tmax'].mean(), inplace=True)
    avg_tmax = df_selected['tmax'].mean()
    
    return avg_tmax

In [ ]:
def get_tmin(dataset):
    df_selected = dataset[((dataset['lon'] <= 265.38175) & 
                           (dataset['lon'] >= 255.94595) &
                           (dataset['lat'] >= 36.99900) &
                           (dataset['lat'] <= 41.00132)) |   
                          #Kansas Area
                          
                          ((dataset['lon'] <= 262.71654) & 
                           (dataset['lon'] >= 256.98720) &
                           (dataset['lat'] <= 36.99900) &
                           (dataset['lat'] >= 33.89811)) |
                          #Oklahoma Area
                          
                          ((dataset['lon'] <= 255.94281) & 
                           (dataset['lon'] >= 247.94240) &
                           (dataset['lat'] <= 48.99742) &
                           (dataset['lat'] >= 44.99552)) |
                          #Montana Area
                          
                          ((dataset['lon'] >= 255.94018) & 
                           (dataset['lon'] <= 257.93342) &
                           (dataset['lat'] <= 43.00957) &
                           (dataset['lat'] >= 41.00132))
                          #Nebraska Area
                          
                         ].reset_index(drop = True)
    
    df_selected['tmin'].fillna(df_selected['tmin'].mean(), inplace=True)
    avg_tmin = df_selected['tmin'].mean()
    
    return avg_tmin

In [ ]:
tmax_list = []
for file in file_path_tmax:
    dataset = Dataset(file)
    for i in data_Preprocessing(dataset):
        tmax_list.append(get_tmax(i))
        print(i)
    print(file)

In [ ]:
tmin_list = []
for file in file_path_tmin:
    dataset = Dataset(file)
    for i in data_Preprocessing(dataset):
        tmin_list.append(get_tmin(i))
        print(i)
    print(file)

In [ ]:
df_tem = pd.DataFrame(list(zip(tmax_list,tmin_list)),
                      columns = ['tmax', 'tmin'])
df_tem

In [ ]:
df_tem.to_csv('temperature.csv')